In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table) 
}))


In [2]:
source('paths.sh')
GBE_IDs <- c('INI50', 'INI21001', 'HC269', 'HC382')


In [3]:
'../20211028_freeze/PRSmap.eval.tsv.gz' %>%
fread %>%
rename_with(function(x){str_replace(x, '#', '')}, starts_with("#")) %>%
select(
    -trait_category, -trait_category_plot,
    -WBtest_P, -WBtest_BYq, -WBtest_is_significant
) %>%
filter(trait %in% GBE_IDs) -> base_results_df


In [4]:
'predictive_performance.tsv' %>%
fread %>%
rename_with(function(x){str_replace(x, '#', '')}, starts_with("#")) %>%
select(-predictors, -response) -> additional_results_df


In [10]:
base_results_df %>%
select(trait, trait_name, family) %>%
unique %>%
left_join(
    bind_rows(
        base_results_df %>%
        select(-trait_name, -family) %>%
        filter(split != 'train_val', model == 'PRS', metric %in% c('r2', 'auc')) %>%
        mutate(genotype = 'with penalty factors'),

        additional_results_df %>%
        filter(split != 'train_val', model == 'PRS', metric %in% c('r2', 'auc')) %>%
        mutate(genotype = 'without penalty factors')

    ),
    by = "trait"
) -> combined_df


In [11]:
get_split_order_df <- function(){
    data.frame(
        split = c(
            'train', 'val', 'test', 'non_british_white',
            's_asian', 'e_asian', 'african'
        ),
        split_plot = c(
            'Training (WB)', 'Validation (WB)', 'Test (WB)',
            'Non-British white', 'South Asian', 'East Asian', 'African'
        ),
        stringsAsFactors=F
    ) %>% mutate(split_order=-1 * 1:n())
}


In [12]:
base_results_df %>%
select(trait, trait_name, family) %>%
unique %>%
left_join(
    combined_df %>%
    filter(split == 'test') %>%
    mutate(genotype = paste0('metric_', genotype)) %>%
    select(trait, `eval`, genotype) %>%
    spread(genotype, `eval`),
    by = "trait"
) %>%
left_join(
    combined_df %>%
    filter(split == 'test') %>%
    mutate(genotype = paste0('n_variables_', genotype)) %>%
    select(trait, n_variables, genotype) %>%
    spread(genotype, n_variables),
    by = "trait"
) -> summary_table_df


In [13]:
summary_table_df %>%
rename('#trait' = 'trait') %>%
fwrite('penalty_factor_performance.tsv', sep='\t', na = "NA", quote=F)


In [23]:
p_binomial <- combined_df %>%
filter(metric == 'auc') %>%
left_join(get_split_order_df(), by='split') %>%
# filter(GBE_ID %in% GBE_IDs, split != 'train_val') %>%
ggplot(aes(x = reorder(split_plot, split_order), y=`eval`, color=genotype, fill=genotype)) +
geom_hline(yintercept = .5, color='black') +
geom_bar(stat='identity', position = "dodge2") +
geom_errorbar(aes(ymin = l_eval, ymax = u_eval), color='black', alpha=1, position = "dodge2") +
coord_flip(ylim = c(.45, NA)) +
theme_bw(base_size=14) +
theme(legend.position='bottom') +
labs(
    title = 'Binary traits (Binomial model)',
    x = 'Hold-out test set',
    y = 'Pedictive performance or snpnet PRS model (AUC)', 
    fill='Prioritization',
    color='Prioritization'
) + 
scale_fill_brewer(palette="Dark2") + 
scale_color_brewer(palette="Dark2") +
facet_wrap(~ trait, nrow = 1)

In [24]:
p_gaussian <- combined_df %>%
filter(metric == 'r2') %>%
left_join(get_split_order_df(), by='split') %>%
# filter(GBE_ID %in% GBE_IDs, split != 'train_val') %>%
ggplot(aes(x = reorder(split_plot, split_order), y=`eval`, color=genotype, fill=genotype)) +
geom_bar(stat='identity', position = "dodge2") +
geom_errorbar(aes(ymin = l_eval, ymax = u_eval), color='black', alpha=1, position = "dodge2") +
coord_flip() +
theme_bw(base_size=14) +
theme(legend.position='bottom') +
labs(
    title = 'Quantitative traits (Gaussian model)',
    x = 'Hold-out test set',
    y = latex2exp::TeX('Pedictive performance or snpnet PRS model (\\textit{R}$^2$)'), 
    fill='Prioritization',
    color='Prioritization'
) + 
scale_fill_brewer(palette="Dark2") + 
scale_color_brewer(palette="Dark2") +
facet_wrap(~ trait_name, nrow = 1)


In [25]:
for(ext in c('png', 'pdf')){ggsave(
    sprintf('penalty_factor_performance.%s', ext),
    gridExtra::arrangeGrob(
        p_binomial,
        p_gaussian,
        ncol=1
    ),
    width=8, height=8
)}


In [26]:
summary_table_df

trait,trait_name,family,metric_with penalty factors,metric_without penalty factors,n_variables_with penalty factors,n_variables_without penalty factors
<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<int>
HC269,High cholesterol,binomial,0.6197173,0.6187414,5987,5506
HC382,Asthma,binomial,0.6166376,0.6169773,6430,6819
INI21001,BMI,gaussian,0.1105250,0.1108556,27126,28667
INI50,Standing height,gaussian,0.1775686,0.1760525,51209,55937
